In [3]:
import os
# Get the list of all files and directories
path = "/home/chowder/Documents/personalData/fitbit/takeout-20240717T222555Z-001/Takeout/Fitbit/Global Export Data"
dir_list = os.listdir(path)

In [5]:
hrFilenames = [x for x in dir_list if x.split("_")[0] == "heart"]
hrFilenames[0]

'heart_rate-2021-11-22.json'

In [6]:
import json
data = json.load(open(path + "/" + hrFilenames[0]))
data[0]

{'dateTime': '11/22/21 17:01:48', 'value': {'bpm': 70, 'confidence': 0}}

In [8]:
import pandas as pd

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)

fitbitHRdf = fitbitHRdf.set_index("sampleDT")





In [10]:
fitbitHRdf.dtypes

confidence    int64
value         int64
dtype: object

In [11]:
fitbitHRdf.to_csv("fitbitHRdf.csv")


In [12]:
fitbitHRdf.to_parquet('fitbitHRdf.parquet.gzip',
              compression='gzip') 

In [7]:
import json
import pandas as pd

columnNames = ["sampleDT", "confidence", "value"]

samplesCount = 0
samplesList = []

for fn in hrFilenames:
    data = json.load(open(path + "/" + fn))
    for row in data:
        samplesList.append([pd.to_datetime(row["dateTime"] + "-0700"), row["value"]["confidence"], row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 100_000 == 0:
            print(samplesCount)




100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1